In [1]:
print("all ok")

all ok


In [2]:
# Pseudo-code for embedding generation
from sentence_transformers import SentenceTransformer
from qdrant_client.models import SparseVector

# Dense Model
dense_model = SentenceTransformer('all-MiniLM-L6-v2')

# Sparse (BM25-like)
def get_sparse_vector(text):
    # This would use a tokenizer and compute term frequencies/BM25 scores
    # For simplicity, a basic TF-IDF-like approach can be implemented or
    # leverage specialized libraries/Qdrant's sparse vector handling.
    # Returns: SparseVector(indices=[...], values=[...])
    pass

processed_data = [] # List of {'text': ..., 'metadata': ...}

for chunk in processed_data:
    chunk['dense_vector'] = dense_model.encode(chunk['text']).tolist()
    chunk['sparse_vector'] = get_sparse_vector(chunk['text'])

/Users/saibhargavrallapalli/Documents/Git/qdrant_dust/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, SparseVectorParams, WalConfig, OptimizersConfig

client = QdrantClient(host="localhost", port=6333)
COLLECTION_NAME = "financial_reports_2025"
VECTOR_DIM = 384 # Dimension of all-MiniLM-L6-v2

# Create the collection with both dense and sparse vectors
client.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config={
        "dense": VectorParams(size=VECTOR_DIM, distance=Distance.COSINE),
    },
    sparse_vectors_config={
        "sparse": SparseVectorParams(
            # BM25-based or custom sparse model configuration
        )
    },
)

/var/folders/n2/2s0tjc891jx80m_1lwcfhm_40000gn/T/ipykernel_88403/1642275295.py:9: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [5]:
from qdrant_client.models import PointStruct

# Assuming 'final_data' is the list of dictionaries from Step 2.2
points = []
for i, item in enumerate(final_data):
    points.append(
        PointStruct(
            id=i,
            vector={
                "dense": item['dense_vector'],
                "sparse": item['sparse_vector'],
            },
            payload=item['metadata']
        )
    )

# Batch upload
client.upsert(
    collection_name=COLLECTION_NAME,
    wait=True,
    points=points
)

NameError: name 'final_data' is not defined

In [7]:
from qdrant_client import QdrantClient
from qdrant_client.models import CollectionStatus

client = QdrantClient(host="localhost", port=6333)
collection_info = client.get_collection("financial_reports_2025")

print(f"Collection Status: {collection_info.status}")
print(f"Total Points Count: {collection_info.points_count}")

Collection Status: green
Total Points Count: 7104
